In [1]:
library(AppliedPredictiveModeling)
data(AlzheimerDisease)
ls()

[1] "diagnosis"  "predictors"

In [7]:
str(predictors)

'data.frame':	333 obs. of  130 variables:
 $ ACE_CD143_Angiotensin_Converti  : num  2 1.56 1.52 1.68 2.4 ...
 $ ACTH_Adrenocorticotropic_Hormon : num  -1.386 -1.386 -1.715 -1.609 -0.968 ...
 $ AXL                             : num  1.098 0.683 -0.145 0.683 0.191 ...
 $ Adiponectin                     : num  -5.36 -5.02 -5.81 -5.12 -4.78 ...
 $ Alpha_1_Antichymotrypsin        : num  1.74 1.46 1.19 1.28 2.13 ...
 $ Alpha_1_Antitrypsin             : num  -12.6 -11.9 -13.6 -15.5 -11.1 ...
 $ Alpha_1_Microglobulin           : num  -2.58 -3.24 -2.88 -3.17 -2.34 ...
 $ Alpha_2_Macroglobulin           : num  -72.7 -154.6 -136.5 -98.4 -144.9 ...
 $ Angiopoietin_2_ANG_2            : num  1.065 0.742 0.833 0.916 0.956 ...
 $ Angiotensinogen                 : num  2.51 2.46 1.98 2.38 2.86 ...
 $ Apolipoprotein_A_IV             : num  -1.43 -1.66 -1.66 -2.12 -1.17 ...
 $ Apolipoprotein_A1               : num  -7.4 -7.05 -7.68 -8.05 -6.73 ...
 $ Apolipoprotein_A2               : num  -0.2614 -0.8675

In [2]:
col_baseline <- c("Genotype", "age", "tau", "p_tau", "Ab_42", "male")
col_newassays <- colnames(predictors)
col_newassays <- col_newassays[!(col_newassays %in% c("Class",col_baseline))]

In [3]:
summary(predictors$Genotype)
predictors$E2 <- predictors$E3 <- predictors$E4 <- 0
predictors$E2[grepl("2",predictors$Genotype)] <- 1
predictors$E3[grepl("3",predictors$Genotype)] <- 1
predictors$E4[grepl("4",predictors$Genotype)] <- 1
genotype <- predictors$Genotype

E2E2 E2E3 E2E4 E3E3 E3E4 E4E4 
   2   37    8  167  106   13

In [4]:
library(caret)
set.seed(730)
trn_id <- createDataPartition(diagnosis,p=0.8,list=F)
df_all <- predictors
df_all$Class <- diagnosis

trn <- df_all[trn_id,]
vld <- df_all[-trn_id,]

col_pred <- names(subset(predictors, select = -Genotype))

Loading required package: lattice
Loading required package: ggplot2


In [5]:
fiveStats <- function(...) c(twoClassSummary(...),defaultSummary(...))

In [6]:
set.seed(104)
index <- createMultiFolds(trn$Class,times=5)

In [7]:
## The candidate set of the number of predictors to evaluate
varSeq <- seq(1,length(col_pred)-1,by=2)

In [8]:
library(doMC)
registerDoMC()

Loading required package: foreach
Loading required package: iterators
Loading required package: parallel


`Forward, Backward, and Stepwise Selection`

In [10]:
initial <- glm(Class~tau + VEGF + E4 + IL_3, data=trn,family = binomial)
initial


Call:  glm(formula = Class ~ tau + VEGF + E4 + IL_3, family = binomial, 
    data = trn)

Coefficients:
(Intercept)          tau         VEGF           E4         IL_3  
     8.5184      -4.2096       0.9848      -0.2150      -0.2102  

Degrees of Freedom: 266 Total (i.e. Null);  262 Residual
Null Deviance:	    313.3 
Residual Deviance: 179.5 	AIC: 189.5

In [12]:
library(MASS)
stepAIC(initial,direction = "both")

Start:  AIC=189.46
Class ~ tau + VEGF + E4 + IL_3

       Df Deviance    AIC
- IL_3  1   179.69 187.69
- E4    1   179.72 187.72
<none>      179.46 189.46
- VEGF  1   242.77 250.77
- tau   1   288.61 296.61

Step:  AIC=187.69
Class ~ tau + VEGF + E4

       Df Deviance    AIC
- E4    1   179.84 185.84
<none>      179.69 187.69
+ IL_3  1   179.46 189.46
- VEGF  1   248.30 254.30
- tau   1   290.05 296.05

Step:  AIC=185.84
Class ~ tau + VEGF

       Df Deviance    AIC
<none>      179.84 185.84
+ E4    1   179.69 187.69
+ IL_3  1   179.72 187.72
- VEGF  1   255.07 259.07
- tau   1   300.69 304.69



Call:  glm(formula = Class ~ tau + VEGF, family = binomial, data = trn)

Coefficients:
(Intercept)          tau         VEGF  
     9.8075      -4.2779       0.9761  

Degrees of Freedom: 266 Total (i.e. Null);  264 Residual
Null Deviance:	    313.3 
Residual Deviance: 179.8 	AIC: 185.8

`Recursive Feature Elimination`

In [13]:
library(caret)
str(rfFuncs)

List of 6
 $ summary   :function (data, lev = NULL, model = NULL)  
 $ fit       :function (x, y, first, last, ...)  
 $ pred      :function (object, x)  
 $ rank      :function (object, x, y)  
 $ selectSize:function (x, metric, maximize)  
 $ selectVar :function (y, size)  


In [14]:
newRF <- rfFuncs
newRF$summary <- fiveStats

ctrl <- rfeControl(method="repeatedcv",repeats=5,
                   verbose=T,functions=newRF,index=index)

set.seed(721)
fit_rf_rfe <- rfe(x=trn[,col_pred],y=trn$Class,
                 sizes=varSeq,metric="ROC",rfeControl=ctrl,ntree=500)
fit_rf_rfe

+(rfe) fit Fold01.Rep1 size: 132 
-(rfe) fit Fold01.Rep1 size: 132 
+(rfe) imp Fold01.Rep1 
-(rfe) imp Fold01.Rep1 
+(rfe) fit Fold01.Rep1 size: 131 
-(rfe) fit Fold01.Rep1 size: 131 
+(rfe) fit Fold01.Rep1 size: 129 
-(rfe) fit Fold01.Rep1 size: 129 
+(rfe) fit Fold01.Rep1 size: 127 
-(rfe) fit Fold01.Rep1 size: 127 
+(rfe) fit Fold01.Rep1 size: 125 
-(rfe) fit Fold01.Rep1 size: 125 
+(rfe) fit Fold01.Rep1 size: 123 
-(rfe) fit Fold01.Rep1 size: 123 
+(rfe) fit Fold01.Rep1 size: 121 
-(rfe) fit Fold01.Rep1 size: 121 
+(rfe) fit Fold01.Rep1 size: 119 
-(rfe) fit Fold01.Rep1 size: 119 
+(rfe) fit Fold01.Rep1 size: 117 
-(rfe) fit Fold01.Rep1 size: 117 
+(rfe) fit Fold01.Rep1 size: 115 
-(rfe) fit Fold01.Rep1 size: 115 
+(rfe) fit Fold01.Rep1 size: 113 
-(rfe) fit Fold01.Rep1 size: 113 
+(rfe) fit Fold01.Rep1 size: 111 
-(rfe) fit Fold01.Rep1 size: 111 
+(rfe) fit Fold01.Rep1 size: 109 
-(rfe) fit Fold01.Rep1 size: 109 
+(rfe) fit Fold01.Rep1 size: 107 
-(rfe) fit Fold01.Rep1 size: 107 


-(rfe) fit Fold02.Rep1 size:  29 
+(rfe) fit Fold02.Rep1 size:  27 
-(rfe) fit Fold02.Rep1 size:  27 
+(rfe) fit Fold02.Rep1 size:  25 
-(rfe) fit Fold02.Rep1 size:  25 
+(rfe) fit Fold02.Rep1 size:  23 
-(rfe) fit Fold02.Rep1 size:  23 
+(rfe) fit Fold02.Rep1 size:  21 
-(rfe) fit Fold02.Rep1 size:  21 
+(rfe) fit Fold02.Rep1 size:  19 
-(rfe) fit Fold02.Rep1 size:  19 
+(rfe) fit Fold02.Rep1 size:  17 
-(rfe) fit Fold02.Rep1 size:  17 
+(rfe) fit Fold02.Rep1 size:  15 
-(rfe) fit Fold02.Rep1 size:  15 
+(rfe) fit Fold02.Rep1 size:  13 
-(rfe) fit Fold02.Rep1 size:  13 
+(rfe) fit Fold02.Rep1 size:  11 
-(rfe) fit Fold02.Rep1 size:  11 
+(rfe) fit Fold02.Rep1 size:   9 
-(rfe) fit Fold02.Rep1 size:   9 
+(rfe) fit Fold02.Rep1 size:   7 
-(rfe) fit Fold02.Rep1 size:   7 
+(rfe) fit Fold02.Rep1 size:   5 
-(rfe) fit Fold02.Rep1 size:   5 
+(rfe) fit Fold02.Rep1 size:   3 
-(rfe) fit Fold02.Rep1 size:   3 
+(rfe) fit Fold02.Rep1 size:   1 
-(rfe) fit Fold02.Rep1 size:   1 
+(rfe) fit Fol

-(rfe) fit Fold04.Rep1 size:  57 
+(rfe) fit Fold04.Rep1 size:  55 
-(rfe) fit Fold04.Rep1 size:  55 
+(rfe) fit Fold04.Rep1 size:  53 
-(rfe) fit Fold04.Rep1 size:  53 
+(rfe) fit Fold04.Rep1 size:  51 
-(rfe) fit Fold04.Rep1 size:  51 
+(rfe) fit Fold04.Rep1 size:  49 
-(rfe) fit Fold04.Rep1 size:  49 
+(rfe) fit Fold04.Rep1 size:  47 
-(rfe) fit Fold04.Rep1 size:  47 
+(rfe) fit Fold04.Rep1 size:  45 
-(rfe) fit Fold04.Rep1 size:  45 
+(rfe) fit Fold04.Rep1 size:  43 
-(rfe) fit Fold04.Rep1 size:  43 
+(rfe) fit Fold04.Rep1 size:  41 
-(rfe) fit Fold04.Rep1 size:  41 
+(rfe) fit Fold04.Rep1 size:  39 
-(rfe) fit Fold04.Rep1 size:  39 
+(rfe) fit Fold04.Rep1 size:  37 
-(rfe) fit Fold04.Rep1 size:  37 
+(rfe) fit Fold04.Rep1 size:  35 
-(rfe) fit Fold04.Rep1 size:  35 
+(rfe) fit Fold04.Rep1 size:  33 
-(rfe) fit Fold04.Rep1 size:  33 
+(rfe) fit Fold04.Rep1 size:  31 
-(rfe) fit Fold04.Rep1 size:  31 
+(rfe) fit Fold04.Rep1 size:  29 
-(rfe) fit Fold04.Rep1 size:  29 
+(rfe) fit Fol

-(rfe) fit Fold06.Rep1 size:  85 
+(rfe) fit Fold06.Rep1 size:  83 
-(rfe) fit Fold06.Rep1 size:  83 
+(rfe) fit Fold06.Rep1 size:  81 
-(rfe) fit Fold06.Rep1 size:  81 
+(rfe) fit Fold06.Rep1 size:  79 
-(rfe) fit Fold06.Rep1 size:  79 
+(rfe) fit Fold06.Rep1 size:  77 
-(rfe) fit Fold06.Rep1 size:  77 
+(rfe) fit Fold06.Rep1 size:  75 
-(rfe) fit Fold06.Rep1 size:  75 
+(rfe) fit Fold06.Rep1 size:  73 
-(rfe) fit Fold06.Rep1 size:  73 
+(rfe) fit Fold06.Rep1 size:  71 
-(rfe) fit Fold06.Rep1 size:  71 
+(rfe) fit Fold06.Rep1 size:  69 
-(rfe) fit Fold06.Rep1 size:  69 
+(rfe) fit Fold06.Rep1 size:  67 
-(rfe) fit Fold06.Rep1 size:  67 
+(rfe) fit Fold06.Rep1 size:  65 
-(rfe) fit Fold06.Rep1 size:  65 
+(rfe) fit Fold06.Rep1 size:  63 
-(rfe) fit Fold06.Rep1 size:  63 
+(rfe) fit Fold06.Rep1 size:  61 
-(rfe) fit Fold06.Rep1 size:  61 
+(rfe) fit Fold06.Rep1 size:  59 
-(rfe) fit Fold06.Rep1 size:  59 
+(rfe) fit Fold06.Rep1 size:  57 
-(rfe) fit Fold06.Rep1 size:  57 
+(rfe) fit Fol

-(rfe) fit Fold08.Rep1 size: 113 
+(rfe) fit Fold08.Rep1 size: 111 
-(rfe) fit Fold08.Rep1 size: 111 
+(rfe) fit Fold08.Rep1 size: 109 
-(rfe) fit Fold08.Rep1 size: 109 
+(rfe) fit Fold08.Rep1 size: 107 
-(rfe) fit Fold08.Rep1 size: 107 
+(rfe) fit Fold08.Rep1 size: 105 
-(rfe) fit Fold08.Rep1 size: 105 
+(rfe) fit Fold08.Rep1 size: 103 
-(rfe) fit Fold08.Rep1 size: 103 
+(rfe) fit Fold08.Rep1 size: 101 
-(rfe) fit Fold08.Rep1 size: 101 
+(rfe) fit Fold08.Rep1 size:  99 
-(rfe) fit Fold08.Rep1 size:  99 
+(rfe) fit Fold08.Rep1 size:  97 
-(rfe) fit Fold08.Rep1 size:  97 
+(rfe) fit Fold08.Rep1 size:  95 
-(rfe) fit Fold08.Rep1 size:  95 
+(rfe) fit Fold08.Rep1 size:  93 
-(rfe) fit Fold08.Rep1 size:  93 
+(rfe) fit Fold08.Rep1 size:  91 
-(rfe) fit Fold08.Rep1 size:  91 
+(rfe) fit Fold08.Rep1 size:  89 
-(rfe) fit Fold08.Rep1 size:  89 
+(rfe) fit Fold08.Rep1 size:  87 
-(rfe) fit Fold08.Rep1 size:  87 
+(rfe) fit Fold08.Rep1 size:  85 
-(rfe) fit Fold08.Rep1 size:  85 
+(rfe) fit Fol

...

-(rfe) fit Fold03.Rep5 size:   3 
+(rfe) fit Fold03.Rep5 size:   1 
-(rfe) fit Fold03.Rep5 size:   1 
+(rfe) fit Fold04.Rep5 size: 132 
-(rfe) fit Fold04.Rep5 size: 132 
+(rfe) imp Fold04.Rep5 
-(rfe) imp Fold04.Rep5 
+(rfe) fit Fold04.Rep5 size: 131 
-(rfe) fit Fold04.Rep5 size: 131 
+(rfe) fit Fold04.Rep5 size: 129 
-(rfe) fit Fold04.Rep5 size: 129 
+(rfe) fit Fold04.Rep5 size: 127 
-(rfe) fit Fold04.Rep5 size: 127 
+(rfe) fit Fold04.Rep5 size: 125 
-(rfe) fit Fold04.Rep5 size: 125 
+(rfe) fit Fold04.Rep5 size: 123 
-(rfe) fit Fold04.Rep5 size: 123 
+(rfe) fit Fold04.Rep5 size: 121 
-(rfe) fit Fold04.Rep5 size: 121 
+(rfe) fit Fold04.Rep5 size: 119 
-(rfe) fit Fold04.Rep5 size: 119 
+(rfe) fit Fold04.Rep5 size: 117 
-(rfe) fit Fold04.Rep5 size: 117 
+(rfe) fit Fold04.Rep5 size: 115 
-(rfe) fit Fold04.Rep5 size: 115 
+(rfe) fit Fold04.Rep5 size: 113 
-(rfe) fit Fold04.Rep5 size: 113 
+(rfe) fit Fold04.Rep5 size: 111 
-(rfe) fit Fold04.Rep5 size: 111 
+(rfe) fit Fold04.Rep5 size: 109 


-(rfe) fit Fold05.Rep5 size:  31 
+(rfe) fit Fold05.Rep5 size:  29 
-(rfe) fit Fold05.Rep5 size:  29 
+(rfe) fit Fold05.Rep5 size:  27 
-(rfe) fit Fold05.Rep5 size:  27 
+(rfe) fit Fold05.Rep5 size:  25 
-(rfe) fit Fold05.Rep5 size:  25 
+(rfe) fit Fold05.Rep5 size:  23 
-(rfe) fit Fold05.Rep5 size:  23 
+(rfe) fit Fold05.Rep5 size:  21 
-(rfe) fit Fold05.Rep5 size:  21 
+(rfe) fit Fold05.Rep5 size:  19 
-(rfe) fit Fold05.Rep5 size:  19 
+(rfe) fit Fold05.Rep5 size:  17 
-(rfe) fit Fold05.Rep5 size:  17 
+(rfe) fit Fold05.Rep5 size:  15 
-(rfe) fit Fold05.Rep5 size:  15 
+(rfe) fit Fold05.Rep5 size:  13 
-(rfe) fit Fold05.Rep5 size:  13 
+(rfe) fit Fold05.Rep5 size:  11 
-(rfe) fit Fold05.Rep5 size:  11 
+(rfe) fit Fold05.Rep5 size:   9 
-(rfe) fit Fold05.Rep5 size:   9 
+(rfe) fit Fold05.Rep5 size:   7 
-(rfe) fit Fold05.Rep5 size:   7 
+(rfe) fit Fold05.Rep5 size:   5 
-(rfe) fit Fold05.Rep5 size:   5 
+(rfe) fit Fold05.Rep5 size:   3 
-(rfe) fit Fold05.Rep5 size:   3 
+(rfe) fit Fol

-(rfe) fit Fold07.Rep5 size:  59 
+(rfe) fit Fold07.Rep5 size:  57 
-(rfe) fit Fold07.Rep5 size:  57 
+(rfe) fit Fold07.Rep5 size:  55 
-(rfe) fit Fold07.Rep5 size:  55 
+(rfe) fit Fold07.Rep5 size:  53 
-(rfe) fit Fold07.Rep5 size:  53 
+(rfe) fit Fold07.Rep5 size:  51 
-(rfe) fit Fold07.Rep5 size:  51 
+(rfe) fit Fold07.Rep5 size:  49 
-(rfe) fit Fold07.Rep5 size:  49 
+(rfe) fit Fold07.Rep5 size:  47 
-(rfe) fit Fold07.Rep5 size:  47 
+(rfe) fit Fold07.Rep5 size:  45 
-(rfe) fit Fold07.Rep5 size:  45 
+(rfe) fit Fold07.Rep5 size:  43 
-(rfe) fit Fold07.Rep5 size:  43 
+(rfe) fit Fold07.Rep5 size:  41 
-(rfe) fit Fold07.Rep5 size:  41 
+(rfe) fit Fold07.Rep5 size:  39 
-(rfe) fit Fold07.Rep5 size:  39 
+(rfe) fit Fold07.Rep5 size:  37 
-(rfe) fit Fold07.Rep5 size:  37 
+(rfe) fit Fold07.Rep5 size:  35 
-(rfe) fit Fold07.Rep5 size:  35 
+(rfe) fit Fold07.Rep5 size:  33 
-(rfe) fit Fold07.Rep5 size:  33 
+(rfe) fit Fold07.Rep5 size:  31 
-(rfe) fit Fold07.Rep5 size:  31 
+(rfe) fit Fol

-(rfe) fit Fold09.Rep5 size:  87 
+(rfe) fit Fold09.Rep5 size:  85 
-(rfe) fit Fold09.Rep5 size:  85 
+(rfe) fit Fold09.Rep5 size:  83 
-(rfe) fit Fold09.Rep5 size:  83 
+(rfe) fit Fold09.Rep5 size:  81 
-(rfe) fit Fold09.Rep5 size:  81 
+(rfe) fit Fold09.Rep5 size:  79 
-(rfe) fit Fold09.Rep5 size:  79 
+(rfe) fit Fold09.Rep5 size:  77 
-(rfe) fit Fold09.Rep5 size:  77 
+(rfe) fit Fold09.Rep5 size:  75 
-(rfe) fit Fold09.Rep5 size:  75 
+(rfe) fit Fold09.Rep5 size:  73 
-(rfe) fit Fold09.Rep5 size:  73 
+(rfe) fit Fold09.Rep5 size:  71 
-(rfe) fit Fold09.Rep5 size:  71 
+(rfe) fit Fold09.Rep5 size:  69 
-(rfe) fit Fold09.Rep5 size:  69 
+(rfe) fit Fold09.Rep5 size:  67 
-(rfe) fit Fold09.Rep5 size:  67 
+(rfe) fit Fold09.Rep5 size:  65 
-(rfe) fit Fold09.Rep5 size:  65 
+(rfe) fit Fold09.Rep5 size:  63 
-(rfe) fit Fold09.Rep5 size:  63 
+(rfe) fit Fold09.Rep5 size:  61 
-(rfe) fit Fold09.Rep5 size:  61 
+(rfe) fit Fold09.Rep5 size:  59 
-(rfe) fit Fold09.Rep5 size:  59 
+(rfe) fit Fol


Recursive feature selection

Outer resampling method: Cross-Validated (10 fold, repeated 5 times) 

Resampling performance over subset size:

 Variables    ROC   Sens   Spec Accuracy  Kappa   ROCSD SensSD  SpecSD
         1 0.7013 0.4718 0.7827   0.6978 0.2521 0.10253 0.1560 0.09849
         3 0.8126 0.5836 0.8848   0.8026 0.4801 0.09958 0.2114 0.08084
         5 0.8625 0.6311 0.9145   0.8374 0.5668 0.08762 0.1834 0.06208
         7 0.8895 0.6639 0.9363   0.8614 0.6282 0.08304 0.1825 0.06254
         9 0.8913 0.6575 0.9518   0.8711 0.6455 0.07760 0.1866 0.05400
        11 0.8967 0.6525 0.9527   0.8705 0.6438 0.07646 0.1789 0.05066
        13 0.8996 0.6857 0.9550   0.8810 0.6748 0.07241 0.1809 0.04760
        15 0.8940 0.6675 0.9569   0.8773 0.6633 0.07944 0.1700 0.04414
        17 0.8969 0.6675 0.9568   0.8772 0.6633 0.07401 0.1713 0.04641
        19 0.8978 0.6614 0.9568   0.8756 0.6587 0.07257 0.1634 0.04441
        21 0.8976 0.6457 0.9619   0.8751 0.6495 0.07536 0.1939 0.04107
     

`Filter Methods`

In [17]:
pScore <- function(x,y)
{
    numX <- length(unique(x))
    if(numX > 2){
        out <- t.test(x~y)$p.value
    }else{
        out <- fisher.test(factor(x),y)$p.value
    }
    out
}

In [18]:
scores <- apply(X=trn[,col_pred],2,FUN=pScore,y=trn$Class)
tail(scores)

p_tau        Ab_42         male           E4           E3           E2 
1.699064e-07 8.952405e-13 1.535628e-02 6.396309e-04 1.978571e-01 1.774673e-01

In [19]:
pCorrection <- function(score,x,y){
    score <- p.adjust(score,"bonferroni")
    keepers <- (score <= 0.05)
    keepers
}

tail(pCorrection(scores))

p_tau Ab_42  male    E4    E3    E2 
 TRUE  TRUE FALSE FALSE FALSE FALSE

In [20]:
str(ldaSBF)

List of 5
 $ summary:function (data, lev = NULL, model = NULL)  
 $ fit    :function (x, y, ...)  
 $ pred   :function (object, x)  
 $ score  :function (x, y)  
 $ filter :function (score, x, y)  


In [23]:
ldaWithPvalues <- ldaSBF
ldaWithPvalues$score <- pScore
ldaWithPvalues$summary <- fiveStats
ldaWithPvalues$filter <- pCorrection

sbfCtrl <- sbfControl(method = "repeatedcv",repeats=5,
                     verbose=T,functions=ldaWithPvalues,index=index)
                      
ldaFilter <- sbf(trn[,col_pred],trn$Class,
                 tol=1.0e-12,sbfControl=sbfCtrl)
                      
                      
ldaFilter


Selection By Filter

Outer resampling method: Cross-Validated (10 fold, repeated 5 times) 

Resampling performance:

    ROC   Sens   Spec Accuracy  Kappa   ROCSD SensSD  SpecSD AccuracySD KappaSD
 0.8563 0.6443 0.9083   0.8361 0.5663 0.07646  0.201 0.06721    0.06283  0.1778

Using the training set, 17 variables were selected:
   Creatine_Kinase_MB, Eotaxin_3, FAS, GRO_alpha, IGF_BP_2...

During resampling, the top 5 selected variables (out of a possible 23):
   Ab_42 (100%), GRO_alpha (100%), MIF (100%), p_tau (100%), tau (100%)

On average, 13.5 variables were selected (min = 9, max = 19)